In [29]:
folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian/permutations/'


In [30]:
# %%bash

! mkdir $folder/barplots
!mkdir $folder/tidy
!mkdir $folder/size
!mkdir $folder/psi
# mkdir $folder/modalities_sizes

In [31]:
base_folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian/permutations'


In [32]:
! mkdir -p $base_folder

In [33]:
%%file ~/randomly_permute_and_calculate_modality.py

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import flotilla
import os
import sys
import anchor

sns.set(style='ticks', context='paper', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

# Ensure the iteration is always at least 1
iteration = max(int(sys.argv[1]), 1)
iteration_str = str(iteration).zfill(4)
suffix = 'iteration_{}'.format(iteration_str)

base_folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian/permutations'

# seed = (sum(ord(c) for c in 'randomly_permute_modalities')/iteration) % 5437
np.random.seed(iteration)

flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects/'

study = flotilla.embark('singlecell_pnm_figure1_supplementary_post_splicing_filtering', flotilla_dir=flotilla_dir)

not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)
splicing_singles_no_outliers = study.splicing.singles.loc[not_outliers]

print splicing_singles_no_outliers.shape
splicing_singles_no_outliers = splicing_singles_no_outliers.groupby(
    study.sample_id_to_phenotype).apply(lambda x: x.dropna(thresh=20, axis=1))
print splicing_singles_no_outliers.shape


# To permute the whole array, have to do it on the flattened values and then reshape them 
# because otherwise numpy only shuffles the rows which is not what we want
permuted_psi = splicing_singles_no_outliers.groupby(study.sample_id_to_phenotype).apply(
    lambda x: pd.DataFrame(np.random.permutation(x.values.flat).reshape(x.shape), index=x.index, columns=x.columns))
permuted_psi.to_csv('{}/psi/{}.csv'.format(base_folder, suffix))

bayesian = anchor.BayesianModalities()
modality_assignments = permuted_psi.groupby(study.sample_id_to_phenotype).apply(bayesian.fit_predict)

modality_assignments[splicing_singles_no_outliers.groupby(study.sample_id_to_phenotype).count() < 20] = np.nan
modalities_tidy = modality_assignments.stack().reset_index()
modalities_tidy = modalities_tidy.rename(columns={'level_1': 'event_name', 0: "modality",})
modalities_tidy.to_csv('{}/tidy/{}.csv'.format(base_folder, suffix), index=False)

sizes = modalities_tidy.groupby(['phenotype', 'modality']).size()
sizes.to_csv('{}/size/{}.csv'.format(base_folder, suffix))


g = anchor.barplot(modalities_tidy, x='phenotype', hue='modality', size=2)
g.savefig('{}/barplots/{}.pdf'.format(base_folder, suffix))

Overwriting /home/obotvinnik/randomly_permute_and_calculate_modality.py


In [36]:
from qtools import Submitter

base_command = 'python ~/randomly_permute_and_calculate_modality.py'

commands = []
for i in range(1000):
    commands.append('{} {}'.format(base_command, i))
Submitter(commands, 'randomly_permute_modality', walltime='1:00:00', array=True, queue='home-scrm')

running 500 tasks as an array-job.
job ID: 4239504
running 500 tasks as an array-job.
job ID: 4239505


In [35]:
! tail *out*

==> randomly_permute_modality.sh.out-1 <==
2016-01-23 10:31:55	Loading expression data
2016-01-23 10:31:55	Initializing expression
2016-01-23 10:31:56	Done initializing expression
2016-01-23 10:31:58	Loading splicing data
2016-01-23 10:31:58	Initializing splicing
2016-01-23 10:31:58	Done initializing splicing
2016-01-23 10:31:58	Successfully initialized a Study object!
(181, 11767)
(181, 8125)
Nodes:        tscc-2-51

==> randomly_permute_modality.sh.out-10 <==
2016-01-23 10:32:06	Loading expression data
2016-01-23 10:32:06	Initializing expression
2016-01-23 10:32:06	Done initializing expression
2016-01-23 10:32:08	Loading splicing data
2016-01-23 10:32:08	Initializing splicing
2016-01-23 10:32:08	Done initializing splicing
2016-01-23 10:32:08	Successfully initialized a Study object!
(181, 11767)
(181, 8125)
Nodes:        tscc-2-50

==> randomly_permute_modality.sh.out-11 <==
2016-01-23 10:32:09	Loading expression data
2016-01-23 10:32:09	Initializing expression
2016-01-23 10:32:09	Don